In [3]:
import os
from dotenv import load_dotenv
import weaviate
import weaviate.classes.query as wvcq
from weaviate.classes.init import Auth
import json

# --- 1. Connection Details ---
load_dotenv()
WEAVIATE_URL = os.environ.get("WEAVIATE_URL")
WEAVIATE_API_KEY = os.environ.get("WEAVIATE_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [ ]:
# --- 2. Connect to Weaviate Cloud (v4 method) ---
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
    headers={"X-OpenAI-Api-Key": OPENAI_API_KEY}
)

# --- 3. Run Queries ---
try:
    products = client.collections.get("EcommerceProducts")

    query_1 = "durable office table"
    print(f'Running Query: "{query_1}"')
    # Query 1: Pure Semantic Search
    response = products.query.near_text(
        query=query_1,
        limit=3,
        return_metadata=wvcq.MetadataQuery(distance=True),
        return_properties=["product_id","product_title", "product_brand", "product_color"]
    )
    for item in response.objects:
        print(json.dumps(item.properties, indent=2))
        print(f"  Distance: {item.metadata.distance:.4f}\n")

    query_2 = "Give me options for different plaques"
    print(f'Running Query: "{query_2}"')
    # Query 1: Pure Semantic Search
    response = products.query.near_text(
        query=query_2,
        limit=5,
        return_metadata=wvcq.MetadataQuery(distance=True),
        return_properties=["product_id","product_title", "product_brand", "product_color"]
    )
    for item in response.objects:
        print(json.dumps(item.properties, indent=2))
        print(f"  Distance: {item.metadata.distance:.4f}\n")
finally:
    client.close()
    print("Connection closed.")

Running Query: "durable office table"
{
  "product_brand": "Lorell",
  "product_id": "B00275YWCS",
  "product_color": "Charcoal",
  "product_title": "Lorell Steel Drawer with Flipper Doors, 48 by 24-Inch, Charcoal"
}
  Distance: 0.4265

{
  "product_brand": "Patavinity",
  "product_id": "B08CDZPKVJ",
  "product_color": "White",
  "product_title": "Patavinity Real Wood Computer Desk, 47in Rustic Wood and Metal Writing Desk, Sturdy PC Study Table for Home Office (47'' W x 23'' D x 29'' H, Pine Wood Desktop with Water-Proof PVC Cover, White)"
}
  Distance: 0.4424

{
  "product_brand": "Topeakmart",
  "product_id": "B07125BRDD",
  "product_color": "Black",
  "product_title": "Topeakmart Mobile Compact Computer Desk Cart for Small Spaces, Work Workstation, Writing Desk Table with Drawers and Printer Shelf on Wheels"
}
  Distance: 0.4432

Running Query: "Give me options for different plaques"
{
  "product_brand": "Crown Awards",
  "product_id": "B07KXYPLY3",
  "product_color": "",
  "product

In [31]:
# --- 2. Connect to Weaviate Cloud (v4 method) ---
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
    headers={"X-OpenAI-Api-Key": OPENAI_API_KEY}
)

# --- 3. Run Queries ---
try:
    products = client.collections.get("EcommerceProducts")

    query_1 = "black hoodies for men while also searching for Nike hoodies"
    filters = [("product_brand", "Nike")]
    # Query 2: Hybrid Search (Semantic + Filter)
    print(f"\n--- 2. Hybrid Search for {query_1} from the {filters[0][0]} '{filters[0][1]}' ---")
    response = products.query.near_text(
        query=query_1,
        limit=5,
        filters=wvcq.Filter.by_property(filters[0][0]).equal(filters[0][1]),
        return_metadata=wvcq.MetadataQuery(distance=True),
        return_properties=["product_title", "product_brand"]
    )
    for item in response.objects:
        print(json.dumps(item.properties, indent=2))
        print(f"  Distance: {item.metadata.distance:.4f}\n")

    query_2 = "find me a pair of sport shoes for ladies while only searching for Nike's shoes and also they should be Black in color."
    # Query 2: Hybrid Search (Semantic + Filter)
    print(f"\n--- 2. Hybrid Search for {query_2} ---")
    response = products.query.near_text(
        query=query_2,
        limit=5,
        return_metadata=wvcq.MetadataQuery(distance=True),
        return_properties=["product_title", "product_brand"]
    )
    for item in response.objects:
        print(json.dumps(item.properties, indent=2))
        print(f"  Distance: {item.metadata.distance:.4f}\n")

    query_3 = "find me a pair of sport shoes for ladies"
    filters = [("product_brand", "Nike"), ("product_color", "Black")] # Also known as Facets
    # Query 2: Hybrid Search (Semantic + Filter)
    print(f"\n--- 2. Hybrid Search for {query_3} from the {filters[0][0]} '{filters[0][1]} and {filters[1][0]} '{filters[1][1]}' ---")
    response = products.query.near_text(
        query=query_3,
        limit=5,
        filters=wvcq.Filter.all_of([wvcq.Filter.by_property(filter[0]).equal(filter[1]) for filter in filters]),
        return_metadata=wvcq.MetadataQuery(distance=True),
        return_properties=["product_title", "product_brand"]
    )
    for item in response.objects:
        print(json.dumps(item.properties, indent=2))
        print(f"  Distance: {item.metadata.distance:.4f}\n")

    # Problematic Query where negative semantic attribute is not considerred properly
    query_4 = "find me a pair of sport shoes for ladies, but not for girls"
    filters = [("product_brand", "Nike"), ("product_color", "Black")] # Also known as Facets
    # Query 2: Hybrid Search (Semantic + Filter)
    print(f"\n--- 2. Hybrid Search for {query_4} from the {filters[0][0]} '{filters[0][1]} and {filters[1][0]} '{filters[1][1]}' ---")
    response = products.query.near_text(
        query=query_4,
        limit=5,
        filters=wvcq.Filter.all_of([wvcq.Filter.by_property(filter[0]).equal(filter[1]) for filter in filters]),
        return_metadata=wvcq.MetadataQuery(distance=True),
        return_properties=["product_title", "product_brand"]
    )
    for item in response.objects:
        print(json.dumps(item.properties, indent=2))
        print(f"  Distance: {item.metadata.distance:.4f}\n")
finally:
    client.close()
    print("Connection closed.")



--- 2. Hybrid Search for black hoodies for men while also searching for Nike hoodies from the product_brand 'Nike' ---
{
  "product_brand": "Nike",
  "product_title": "Nike Men's Sportswear Club Fleece Full Zip Hoodie, Fleece Zip-Up Hoodie Men, Black/Black/White, L"
}
  Distance: 0.4140

{
  "product_brand": "Nike",
  "product_title": "Nike Women's Sportswear Gym Vintage Full-Zip Hoodie (2X, Black/Sail)"
}
  Distance: 0.4829

{
  "product_brand": "Nike",
  "product_title": "Nike Women's NSW Fleece Hoodie Varsity, Echo Pink/Echo Pink/White, Large"
}
  Distance: 0.4861

{
  "product_brand": "Nike",
  "product_title": "Nike NFL Dallas Cowboys Boys Hoodies Sweatshirt Kangaroo Pocket Hoodie (Large, Navy)"
}
  Distance: 0.4900

{
  "product_brand": "Nike",
  "product_title": "Nike Women's NSW Fleece Hoodie Full Zip Varsity, Echo Pink/Echo Pink/White, Medium"
}
  Distance: 0.5077


--- 2. Hybrid Search for find me a pair of sport shoes for ladies while only searching for Nike's shoes and als

In [34]:
# --- 2. Connect to Weaviate Cloud (v4 method) ---
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
    headers={"X-OpenAI-Api-Key": OPENAI_API_KEY}
)

# --- 3. Run Queries ---
try:
    products = client.collections.get("EcommerceProducts")

    query_1 = "black hoodies for men"
    # Query 3: Simple Retrieval with Filter (No Semantic Search) -> Keyword Search
    print(f"\n--- 2. Keyword Search for {query_1} ---")
    response = products.query.bm25(
        query=query_1,
        limit=5,
        return_metadata=wvcq.MetadataQuery(score=True),
        operator=wvcq.BM25Operator.or_(minimum_match=2),
        query_properties=["product_title"],
        return_properties=["product_title", "product_brand"]
    )
    for item in response.objects:
        print(json.dumps(item.properties, indent=2))
        print(f"  Score: {item.metadata.score:.4f}\n")
finally:
    client.close()
    print("Connection closed.")


--- 2. Keyword Search for black hoodies for men ---
{
  "product_brand": "Tstars",
  "product_title": "The Missing Piece Pizza & Slice - His and Her Hoodies - Matching Couple Hoodies Men's Black Large / Women Black Small"
}
  Score: 6.6529

{
  "product_brand": "Champion",
  "product_title": "Big & Tall Sweatshirt For Men Embroidered Pullover Hoodies Black 4X"
}
  Score: 6.4494

{
  "product_brand": "Jerzees",
  "product_title": "Jerzees Men's NuBlend -Fleece -Sweatshirts & -Hoodies, Full Zip-Black, Medium"
}
  Score: 6.2690

{
  "product_brand": "Tess Maltz",
  "product_title": "Tess Maltz Custom Cotton Hoodies Sweatshirt Personalized Sweater Sport Comfortable Men Black"
}
  Score: 6.0984

{
  "product_brand": "Champion",
  "product_title": "Champion Hoodie, Big and Tall Hoodies for Men, Mens Hoodies, Zip Up Hoodie"
}
  Score: 6.0590

Connection closed.
